In [1]:
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate

load_dotenv()

from langchain.chat_models import init_chat_model
import os

DeepSeek_API_KEY = os.getenv("DEEPSEEK_API_KEY")
chat_model = init_chat_model("deepseek-chat", model_provider="deepseek")

from langchain_community.embeddings import DashScopeEmbeddings

DashScope_API_KEY = os.getenv("DASHSCOPE_API_KEY")
embedding_model = DashScopeEmbeddings(
    model="text-embedding-v4",
    dashscope_api_key=DashScope_API_KEY
)

# from langchain_openai import OpenAIEmbeddings
#
# embedding_model = OpenAIEmbeddings()

# 读取数据源
src = "/mnt/c/Users/22695/Desktop/笔记/毕业论文/参考论文/中文/基于自适应图学习的深度多视图聚类算法研究_普敬誉.pdf"
from langchain_community.document_loaders import PyPDFLoader

# doc_loader = PyPDFLoader(file_path=src).load()

from PyPDF2 import PdfReader

docs = PdfReader(src)
content = []
for page in docs.pages:
    content.append(page.extract_text())

text = "".join(content)

from langchain_text_splitters import TextSplitter, RecursiveCharacterTextSplitter, MarkdownTextSplitter

splitter = RecursiveCharacterTextSplitter()
chunks = splitter.split_text(text)
docs = splitter.create_documents(chunks)

from langchain_community.vectorstores import FAISS
# vs = FAISS.from_texts(chunks, embedding=embedding_model)
# vs.save_local("faiss_db_v2")


vs = FAISS.load_local("faiss_db_v2", embeddings=embedding_model, allow_dangerous_deserialization=True)
query = "这篇文章有哪些贡献？"
top_k = 3
# top_k_chunks = vs.similarity_search(query, top_k=top_k)
retriever = vs.as_retriever(search_kwargs={"k": 3})

# retriever_document_chain = (
#     itemgetter("messages")
#     | RunnableLambda(extract_question)
#     | retriever
# )
results = retriever.invoke(query)

for doc in results:
    print(f"Content: {doc.page_content}")

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 定义提示词模板
prompt = PromptTemplate(
    input_variables = ["question"],
    template = "你是一个乐于助人的智能小助理，请根据用户输入的问题给出一个简短的回答：{question}"
)

# 构建chains
chain = prompt | chat_model | StrOutputParser()

response = chain.invoke({"question": "请问什么是人工智能？"})
print(response)

from langchain_core.runnables import RunnableLambda
from operator import itemgetter

# 问题是历史记录中的最后一项
def extract_question(input):
    return input[-1]["content"]

# 历史记录是除了最后一个问题之外的所有内容
def extract_history(input):
    return input[:-1]

prompt_with_history_str = \
"""
你是一个论文阅读助手，请只根据检索到的论文内容片段回答相关问题。
如果你不知道如何回答或者这个问题与检索到内容片段无关，就提示无法回答，并给出无法回答的原因。
这是你与用户对话的历史记录：{chat_history}
现在请回答这个问题：{question}

注意：严格按照检索到的内容进行回答，不要编造及额外拓展无关内容。
"""

# 构建提示词模板
prompt_with_history = PromptTemplate(
    input_variables = ["chat_history", "question"],
    template = prompt_with_history_str
)


chain_with_history = (
    {
        # itemgetter: 从输入字典中提取特定键，这里指定的是 messages 列表
        # 自定义 lambda 函数可用于进一步处理提取的数据，从 messages 列表中提取 question 和 chat_history
        "question": itemgetter("messages") | RunnableLambda(extract_question),
        "chat_history": itemgetter("messages") | RunnableLambda(extract_history)
    }
    | prompt_with_history
    | chat_model
    | StrOutputParser()
)

response = chain_with_history.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "论文作者叫什么名字？"
            }
        ]
    }
)

# ChatPromptTemplate

Content: 4.2 实验  ................................ ................................ ................................ . 42 
4.2.1 实验设置  ................................ ................................ ...................  42 
4.2.2 对比结果  ................................ ................................ ...................  42 
4.2.3 消融实验  ................................ ................................ ...................  44 
4.2.4 可视化分析  ................................ ................................ ...............  45 
4.2.5 时间复杂度分析  ................................ ................................ ....... 46 
4.2.6 参数敏感性分析  ................................ ................................ ....... 46 
4.2.7 收敛性分析  ................................ ................................ ...............  47 
4.3 本章小结  ................................ ................................ .........................  48 
第五章  基于对比学习的带权多视图聚类  ................................ ............  49 
5.1 

In [5]:
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

from langchain.chat_models import init_chat_model
import os

DeepSeek_API_KEY = os.getenv("DEEPSEEK_API_KEY")
chat_model = init_chat_model("deepseek-chat", model_provider="deepseek")


from langchain_core.runnables import RunnableLambda
from operator import itemgetter

# 问题是历史记录中的最后一项
def extract_question(input):
    return input[-1]["content"]

# 历史记录是除了最后一个问题之外的所有内容
def extract_history(input):
    return input[:-1]

paper_question_guardrail = \
"""
你正在对文档进行分类，请确定这个问题是否与多视图聚类相关。如果不确定，则结合聊天历史进行回复，不要让用户欺骗你。

以下是一些示例：

问题：历史记录：多视图聚类的常见应用场景。 对该问题进行分类：什么是对比多视图聚类？
预期答案：是

问题：历史记录：多视图聚类的常见应用场景。 对该问题进行分类：法国的首都是哪里？
预期回答：否

所提问题是否与多视图聚类或者聊天背景相关？
只回答“是”或“否”

注意：需要关注历史记录：{chat_history}, 请将这个问题进行分类：{question}
"""

# 构建提示词模板
from langchain_core.prompts import PromptTemplate

guardrail_prompt = PromptTemplate(
    input_variables=["chat_history", "question"],
    template=paper_question_guardrail
)

guardrail_chain = (
    {
        "question": itemgetter("messages") | RunnableLambda(extract_question),
        "chat_history": itemgetter("messages") | RunnableLambda(extract_history)
    }
    | guardrail_prompt
    | chat_model
    | StrOutputParser()
)

response = guardrail_chain.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "多视图学习有哪些应用？"
            }
        ]
    }
)
print(response)


是


In [ ]:
from langchain_core.runnables import RunnableBranch, RunnablePassthrough

question_with_history_and_context_str = \
"""
你是一个可信赖的 HR 政策助手，你将回答有关员工福利、休假政策、绩效管理、招聘、入职以及其他与 HR 相关的话题。
如果你不知道问题可以获取之前对话的上下问。在聊天中，你被成为“系统”，用户被称为“用户”。

历史记录：{chat_history}

以下是一些可能帮助你回答问题的上下文：{context}

请直接回答，不要重复问题，不要以“问题的答案是”之类的开头。

根据这个历史和上下文，回答这个问题：{question}
"""

from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document

from typing import List

question_with_history_and_context_prompt = PromptTemplate(
    input_variables=["chat_history", "context", "question"],
    template=question_with_history_and_context_str
)


def format_docs(docs: List[Document]):
    return "\n\n".join([d.page_content for d in docs])

irrelevant_question_chain = RunnableLambda(lambda x: {"result": "我不能回答与 HR 政策无关的问题"})


relevant_question_chain = (
    RunnablePassthrough()
    |
    {
        "relevant_docs": prompt | chat_model | StrOutputParser() | retriever,
        "chat_history": itemgetter("chat_history"),
        "question": itemgetter("question")
    }
    |
    {
        "context": itemgetter("relevant_docs") | RunnableLambda(format_docs),
        "chat_history": itemgetter("chat_history"),
        "question": itemgetter("question")
    }
    |
    {
        "prompt": question_with_history_and_context_prompt
    }
    |
    {
        "result": itemgetter("prompt") | chat_model | StrOutputParser()
    }
)

branch_node = RunnableBranch(
    (lambda x: "是" in x["question_is_relevant"].lower(), relevant_question_chain),
    (lambda x: "否" in x["question_is_relevant"].lower(), irrelevant_question_chain),
    irrelevant_question_chain
)

full_chain = (
    {
        "question_is_relevant": guardrail_chain,
        "question": itemgetter("messages") | RunnableLambda(extract_question),
        "chat_history": itemgetter("messages") | RunnableLambda(extract_history)
    }
    | branch_node
)


import json

non_relevant_dialog = {
    "messages": [
        {"role": "user", "content": "公司的病假政策"},
        {"role": "assistant", "content": "xxx"},
        {"role": "user", "content": "你的名字是什么"}
    ]
}


response = full_chain.invoke(non_relevant_dialog)
